<a href="https://colab.research.google.com/github/fjadidi2001/Machine_Learning_Journey/blob/main/Predict_ATP_tennis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import random
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn import utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier
import shap
import kagglehub
import os


# Download latest version
path = kagglehub.dataset_download("anupangadi/tennis-players-ranks-prediction-using-atp-elo")

print("Path to dataset files:", path)

# Find the CSV file within the downloaded directory
for filename in os.listdir(path):
    if filename.endswith(".csv"):
        csv_file = os.path.join(path, filename)
        break  # Stop searching after finding the first CSV file

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file)

# Display some info about the loaded dataset
print(df.head())


Path to dataset files: /root/.cache/kagglehub/datasets/anupangadi/tennis-players-ranks-prediction-using-atp-elo/versions/1
             Player   Age     Elo HardRaw ClayRaw GrassRaw  \
0    Novak Djokovic  34.5  2185.2    2068  2016.3   1942.4   
1   Daniil Medvedev  25.9  2166.2  2095.3  1714.1   1723.4   
2  Alexander Zverev  24.7  2141.3  2056.7  1999.1   1671.4   
3     Roger Federer  39.9  2043.0  1936.7  1749.6   1813.4   
4    Carlos Alcaraz  18.5  2029.5  1906.3  1834.4   1441.4   

   hard court elo rating  clay-court elo rating  grass-court elo rating  \
0                 2126.6                 2100.8                  2063.8   
1                 2130.7                 1940.1                  1944.8   
2                 2099.0                 2070.2                  1906.4   
3                 1989.8                 1896.3                  1928.2   
4                 1967.9                 1932.0                  1735.5   

              Peak Match  Peak Age  Peak Elo Gender  

In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Player                  783 non-null    object 
 1   Age                     783 non-null    float64
 2   Elo                     783 non-null    float64
 3   HardRaw                 783 non-null    object 
 4   ClayRaw                 783 non-null    object 
 5   GrassRaw                783 non-null    object 
 6   hard court elo rating   783 non-null    float64
 7   clay-court elo rating   783 non-null    float64
 8   grass-court elo rating  783 non-null    float64
 9   Peak Match              783 non-null    object 
 10  Peak Age                783 non-null    float64
 11  Peak Elo                783 non-null    float64
 12  Gender                  783 non-null    object 
 13  Rank                    783 non-null    int64  
dtypes: float64(7), int64(1), object(6)
memory 